In [ ]:
# conda activate xbert-env
#!pip install sentence-transformers
!pip install faiss-gpu

     |████████████████▊               | 47.0 MB 7.8 MB/s eta 0:00:06

In [2]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pandas

  Using cached pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)


In [3]:
import pandas as pd
df= pd.read_csv('wiki_movie_plots_deduped.csv')

In [17]:
import os
from sentence_transformers import SentenceTransformer
os.environ['TORCH_HOME'] = 'models/' #setting the environment variable

model = SentenceTransformer('models/sentence_transformers/sbert.net_models_msmarco-distilbert-base-dot-prod-v3').to('cuda')

In [9]:
import faiss
encoded_data = model.encode(df.Plot.tolist())

In [10]:
import numpy as np
import faiss

encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'movie_plot.index')

In [11]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['Title'] = info['Title']
    meta_dict['Plot'] = info['Plot'][:500]
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [13]:
from pprint import pprint
import time
query="Artificial Intelligence based action movie"
results=search(query, top_k=10, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 0.030264854431152344


	 {'Title': 'The Cape Canaveral Monsters', 'Plot': "As the film opens, two mysterious white circles are moving about on a solid black background. As they move, a woman's voice says they must obtain human bodies to carry out their mission.\r\nLive action then begins with a man and woman leaving a beach near Cape Canaveral, Florida. The circles descend on them, causing their car to crash. Both are killed. But their bodies suddenly jerk back to life as they're taken over by the white circles, which are actually extraterrestrials. The woman's face is b"}
	 {'Title': 'Small Soldiers', 'Plot': 'Top defense contractor GloboTech Industries acquires the Heartland Toy Company and as part of the move, Globotech CEO Gil Mars tells Heartland toy designers Larry Benson and Irwin Wayfair to develop actual live-action toys capable of "playing back". Mars selects Larry\'s antagonistic, action figures, the Commando Elite, for the project and Irwin\'s pr

In [12]:
from pprint import pprint
import time
query="Artificial Intelligence based action movie"
results=search(query, top_k=5, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 0.010796546936035156


	 {'Title': 'The Cape Canaveral Monsters', 'Plot': "As the film opens, two mysterious white circles are moving about on a solid black background. As they move, a woman's voice says they must obtain human bodies to carry out their mission.\r\nLive action then begins with a man and woman leaving a beach near Cape Canaveral, Florida. The circles descend on them, causing their car to crash. Both are killed. But their bodies suddenly jerk back to life as they're taken over by the white circles, which are actually extraterrestrials. The woman's face is b"}
	 {'Title': 'Small Soldiers', 'Plot': 'Top defense contractor GloboTech Industries acquires the Heartland Toy Company and as part of the move, Globotech CEO Gil Mars tells Heartland toy designers Larry Benson and Irwin Wayfair to develop actual live-action toys capable of "playing back". Mars selects Larry\'s antagonistic, action figures, the Commando Elite, for the project and Irwin\'s pr